In [2]:
import matplotlib.pyplot as plt
import matplotlib.colors
import numpy as np
import os
import cv2
import pandas as pd
import argparse
import shutil
import seaborn as sns
import json
import h5py
import glob
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve, accuracy_score, precision_score, recall_score

## Get predictions

In [3]:
unbeat_steed = "/home/mathiane/LNENWork/data_RoFormerMIL_CaA1CaA2_norm/dataRoFormer/results/task_4_carcinoids_CaA1_CaA2/CaA1Ca2_unbalanced_norm/label_frac=100/BTRoPEAMIL_s_pixel/training/2024-04-11_21-24-30"
stylish_mule = "/home/mathiane/LNENWork/data_RoFormerMIL_CaA1CaA2_norm/dataRoFormer/results/task_4_carcinoids_CaA1_CaA2/CaA1Ca2_unbalanced_norm/label_frac=100/BTRoPEAMIL_s_pixel/training/2024-04-15_19-49-20"
unbeat_steed_eval = "/home/mathiane/LNENWork/data_RoFormerMIL_CaA1CaA2_norm/dataRoFormer/results/task_4_carcinoids_CaA1_CaA2/CaA1Ca2_unbalanced_norm/label_frac=100/BTRoPEAMIL_s_pixel/eval/2024-04-16_11-21-59"
list_unbeat_steed = glob.glob(f"{unbeat_steed}/patients_preds.parquet/*/*/*/*.parquet")
list_unbeat_eval = glob.glob(f"{unbeat_steed_eval}/patients_preds.parquet/*/*/*.parquet")
list_stylish_mule = glob.glob(f"{stylish_mule}/patients_preds.parquet/*/*/*/*.parquet")

In [4]:
list_res_test = list_unbeat_steed +list_stylish_mule + list_unbeat_eval

In [5]:
df_res_c = pd.read_parquet(list_unbeat_eval[0])
df_res_c

,slide_id,probs,labels
0,TNE0009,"[0.6791048049926758, 0.32089516520500183]",0


In [6]:
df_res_c["probs"].values

array([array([0.6791048 , 0.32089517])], dtype=object)

In [7]:
df_res_c["probs"].values[0][0]

0.6791048049926758

In [8]:
c = 0 
for path_res in list_res_test:
    if path_res not in list_unbeat_eval:
        df_res_c = pd.read_parquet(path_res)
        df_res_c["A1"] = df_res_c["probas"].values[0][0]
        df_res_c["A2"] = df_res_c["probas"].values[0][1]
        df_res_c = df_res_c.dropna(subset=["A1"])
    else:
        df_res_c_tmp = pd.read_parquet(path_res)
        ## Get pred -
        preds = np.argmax(df_res_c_tmp["probs"].values[0,])
        slide_id = df_res_c_tmp["slide_id"].values[0]
        labels = df_res_c_tmp["labels"].values[0]

        A1 = df_res_c_tmp["probs"].values[0][0]
        A2 = df_res_c_tmp["probs"].values[0][1]
        print(df_res_c_tmp)
        
        df_res_c = pd.DataFrame(data = {"slide_id" : [slide_id],
                                       "preds" : [preds],
                                       "labels" :[labels],
                                       "probas" :  df_res_c_tmp["probs"].values,
                                       "A1":A1,
                                       "A2": A2
                                       })
    if c == 0:
        df_res = df_res_c     
    else:
        df_res = pd.concat([df_res, df_res_c])
    c +=1
df_res.head()

  slide_id                                      probs  labels
0  TNE0009  [0.6791048049926758, 0.32089516520500183]       0


,slide_id,preds,labels,probas,A1,A2
0,TNE0954,1,1,"[0.16779744625091553, 0.8322026133537292]",0.167797,0.832203
0,TNE0953,1,1,"[0.07978201657533646, 0.9202179908752441]",0.079782,0.920218
0,TNE0883,0,0,"[0.6546739935874939, 0.3453260362148285]",0.654674,0.345326
0,TNE0058,1,0,"[0.26412105560302734, 0.7358789443969727]",0.264121,0.735879
0,TNE0987,0,1,"[0.8802319765090942, 0.11976800858974457]",0.880232,0.119768


In [9]:
df_res.shape

(130, 6)

In [12]:
df_res["slide_id"].tolist()

['TNE0954',
 'TNE0953',
 'TNE0883',
 'TNE0058',
 'TNE0987',
 'TNE1350',
 'TNE1508',
 'TNE1418',
 'TNE0986',
 'TNE1082',
 'TNE0997',
 'TNE1102',
 'TNE0898',
 'TNE0072',
 'TNE0985',
 'TNE1001',
 'TNE1008',
 'TNE0229',
 'TNE0797',
 'TNE0340',
 'TNE1416',
 'TNE1089',
 'TNE0341',
 'TNE0969',
 'TNE0031',
 'TNE0881',
 'TNE1448',
 'TNE0228',
 'TNE1794',
 'TNE0889',
 'TNE1430',
 'TNE0993',
 'TNE0080',
 'TNE1086',
 'TNE1080',
 'TNE0880',
 'TNE0879',
 'TNE0001',
 'TNE0075',
 'TNE0956',
 'TNE1011',
 'TNE1414',
 'TNE0983',
 'TNE1426',
 'TNE0231',
 'TNE0965',
 'TNE1442',
 'TNE1409',
 'TNE0979',
 'TNE0975',
 'TNE1458',
 'TNE1407',
 'TNE1435',
 'TNE1404',
 'TNE0893',
 'TNE0884',
 'TNE0873',
 'TNE0081',
 'TNE0974',
 'TNE0077',
 'TNE1371',
 'TNE0988',
 'TNE1405',
 'TNE1078',
 'TNE0809',
 'TNE0181',
 'TNE1367',
 'TNE0959',
 'TNE0962',
 'TNE0079',
 'TNE0232',
 'TNE0003',
 'TNE0406',
 'TNE0886',
 'TNE0952',
 'TNE1075',
 'TNE0882',
 'TNE1098',
 'TNE1097',
 'TNE0008',
 'TNE1077',
 'TNE0404',
 'TNE0387',
 'TN

In [10]:
df_res[df_res["slide_id"] == "TNE0058"]

,slide_id,preds,labels,probas,A1,A2
0,TNE0058,1,0,"[0.26412105560302734, 0.7358789443969727]",0.264121,0.735879


In [93]:
df_res = df_res.dropna(subset=["A1"])

In [94]:
df_res[df_res['A1'].isnull()]

,slide_id,preds,labels,probas,A1,A2


In [95]:
print(precision_score(df_res['labels'], df_res['preds']))
print(recall_score(df_res['labels'], df_res['preds']))
print(accuracy_score(df_res['labels'], df_res['preds']))
print(average_precision_score(df_res['labels'], df_res['preds']))
print(roc_auc_score(df_res['labels'], df_res['A2']))

0.85
0.85
0.8615384615384616
0.7917307692307691
0.9092857142857144


In [96]:
labels_names =  [  ]
for ele in df_res["labels"].values:
    if ele == 0:
        labels_names.append("CaA1")
    else:
        labels_names.append("CaA2")
        

In [99]:
df_res=  df_res.assign(labels_names = labels_names)

In [101]:
df_res.to_csv("/home/mathiane/LNENWork/data_RoFormerMIL_CaA1CaA2_norm/dataRoFormer/script_analysis/unbeat_steed_2024_04_11_21-24/predictions_all_test_folds.csv", index=False)

## Get all attention scores

In [105]:
attn_scores_unbeat_steed = glob.glob(f"{unbeat_steed}/attention_scores/*/*.h5")
attn_scores_stylish_mule = glob.glob(f"{stylish_mule}/attention_scores/*/*.h5")
attn_scores_unbeat_steed_eval = glob.glob(f"{unbeat_steed_eval}/attention_scores/*.h5")

In [107]:
attn_scores_fname  =  attn_scores_unbeat_steed + attn_scores_stylish_mule + attn_scores_unbeat_steed_eval

In [117]:
RootCoords = "/home/mathiane/LNENWork/data_RoFormerMIL_CaA1CaA2_norm/coords_table"
Coords_table_A1 = glob.glob(f"{RootCoords}/*/*coords.csv")

In [145]:
rootoutdir = "/home/mathiane/LNENWork/data_RoFormerMIL_CaA1CaA2_norm/dataRoFormer/script_analysis/unbeat_steed_2024_04_11_21-24"
outfolder = "attn_maps_formated"
os.makedirs(os.path.join(rootoutdir, outfolder), exist_ok=True)
for fname in attn_scores_fname:
    attn_scores_file = h5py.File(fname, 'r')
    attn_scores_coords = attn_scores_file.get("coords").value
    attn_scores = attn_scores_file.get("attention_scores").value
    attn_scores_a1 = attn_scores[0,:,:].transpose().mean(axis=1)
    attn_scores_a2 = attn_scores[1,:,:].transpose().mean(axis=1)
    x_rank = attn_scores_coords[:,0]
    y_rank = attn_scores_coords[:,1]
    attn_scores_df_c = pd.DataFrame(data={"attn_scores_a1":attn_scores_a1,
                                          "attn_scores_a2": attn_scores_a2,
                                          "x_rank":x_rank,
                                          "y_rank": y_rank})
    ## Get coords table
    attn_scores_samples_id = fname.split("/")[-1].split(".")[0]
    for coords_fname in Coords_table_A1:
        if coords_fname.find(attn_scores_samples_id) != -1:
            coords_df = pd.read_csv(coords_fname)
    attn_scores_to_export = pd.merge(attn_scores_df_c,coords_df,  how='inner', on=['x_rank','y_rank'])
    attn_scores_to_export = attn_scores_to_export.assign(tne_id=np.repeat(attn_scores_samples_id, attn_scores_to_export.shape[0]).tolist())
    
    attn_scores_to_export.to_csv(f"{rootoutdir}/{outfolder}/AttnScores_{attn_scores_samples_id}.csv", index=False)

/home/mathiane/miniconda3/envs/HaloAE/lib/python3.6/site-packages/ipykernel_launcher.py:6: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  
/home/mathiane/miniconda3/envs/HaloAE/lib/python3.6/site-packages/ipykernel_launcher.py:7: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  import sys


In [142]:
attn_scores_to_export.shape

(16171, 8)

,attn_scores_a1,attn_scores_a2,x_rank,y_rank,img_id_c,x,y
0,0.000059,0.000075,0,33,TNE0058_3457_3073,3457,3073
1,0.000060,0.000075,1,33,TNE0058_3841_3073,3841,3073
2,0.000064,0.000052,1,78,TNE0058_3841_3457,3841,3457
3,0.000055,0.000079,3,2,TNE0058_4225_2689,4225,2689
4,0.000061,0.000076,3,33,TNE0058_4225_3073,4225,3073
...,...,...,...,...,...,...,...
16166,0.000057,0.000078,16164,12490,TNE0058_96385_50305,96385,50305
16167,0.000054,0.000068,16164,12564,TNE0058_96385_50689,96385,50689
16168,0.000054,0.000083,16164,12723,TNE0058_96385_51457,96385,51457
16169,0.000052,0.000072,16164,12802,TNE0058_96385_51841,96385,51841


,attn_scores_a1,attn_scores_a2,x_rank,y_rank,tne_id
0,0.000059,0.000075,0,33,TNE0058
1,0.000060,0.000075,1,33,TNE0058
2,0.000064,0.000052,1,78,TNE0058
3,0.000055,0.000079,3,2,TNE0058
4,0.000061,0.000076,3,33,TNE0058
...,...,...,...,...,...
16166,0.000057,0.000078,16164,12490,TNE0058
16167,0.000054,0.000068,16164,12564,TNE0058
16168,0.000054,0.000083,16164,12723,TNE0058
16169,0.000052,0.000072,16164,12802,TNE0058


In [137]:
np.repeat(attn_scores_samples_id, attn_scores_df_c.shape[0])

array(['TNE0058', 'TNE0058', 'TNE0058', ..., 'TNE0058', 'TNE0058',
       'TNE0058'], dtype='<U7')